# ReAct

In [28]:
import os
from dotenv import load_dotenv

load_dotenv()
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

In [29]:
from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType
from langchain.chat_models import ChatOpenAI
from langchain.utilities import SerpAPIWrapper
from langchain_experimental.tools import PythonREPLTool

In [32]:
llm = ChatOpenAI(model="gpt-4-0613")

tools = [
    Tool(
        name="Search",
        description="Useful if you need to search about current events",
        func=SerpAPIWrapper(params={"gl": "de", "hl": "de"}).run
    ),
    PythonREPLTool()
]

agent_chain = initialize_agent(
    tools, 
    llm, 
    agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
    max_execution_time=120
)

In [33]:
agent_chain.invoke("Wann beginnt der Adventsmarkt im St. Katherinenspital Regensburg 2023? Wie viele Tage sind es bis dahin?")



> Entering new AgentExecutor chain...
Thought: I don't know off the top of my head when the Advent market at St. Katherinenspital Regensburg starts in 2023. I'll need to look that up. 

Action:
```
{
  "action": "Search",
  "action_input": "St. Katherinenspital Regensburg Adventsmarkt 2023 start date"
}
```
Observation: ['spital-adventsmarkt 2023. auf dem Spitalanger (im Sommer: Spital-Biergarten). Kontakt für Fragen zum Adventsmarkt 2023: ... 93059 Regensburg. Telefon: +49 (0)941 ...']
Thought:The search results didn't provide the exact date for the Advent market at St. Katherinenspital Regensburg in 2023. I'll perform another search to try to find more specific information.

Action:
```
{
  "action": "Search",
  "action_input": "Startdatum des Adventsmarkts im St. Katherinenspital Regensburg 2023"
}
```
Observation: Katharinenspital 27.11. - 23.12.2023. Auch in diesem Jahr verwandelt sich der Spitalgarten in Stadtamhof wieder in einen adventlichen Markt mit besonderem Zauber.
Thoug

{'input': 'Wann beginnt der Adventsmarkt im St. Katherinenspital Regensburg 2023? Wie viele Tage sind es bis dahin?',
 'output': 'Der Adventsmarkt im St. Katherinenspital Regensburg beginnt am 27. November 2023. Es sind 27 Tage bis dahin.'}

In [ ]:
agent_chain.invoke("Johann Schenkl hält einen Vortrag im Rahmen der der Hacky Hour. Was ist das Thema der Veranstaltung und in wie vielen Tagen findet sie statt?")

In [44]:
for i in agent_chain.agent.llm_chain.prompt.messages:
    print(i.prompt.template)

Answer the following questions as best you can. You have access to the following tools:

Search: Useful if you need to search about current events
Python_REPL: A Python shell. Use this to execute python commands. Input should be a valid python command. If you want to see the output of a value, you should print it out with `print(...)`.

The way you use the tools is by specifying a json blob.
Specifically, this json should have a `action` key (with the name of the tool to use) and a `action_input` key (with the input to the tool going here).

The only values that should be in the "action" field are: Search, Python_REPL

The $JSON_BLOB should only contain a SINGLE action, do NOT return a list of multiple actions. Here is an example of a valid $JSON_BLOB:

```
{{
  "action": $TOOL_NAME,
  "action_input": $INPUT
}}
```

ALWAYS use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action:
```
$JSON_BLOB
```
Observation: the